<a href="https://www.kaggle.com/code/sriramgugulothu/dl-assignment3?scriptVersionId=177557141" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
from torch import nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import copy
from torch.utils.data import Dataset, DataLoader
import random

In [33]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
wandb.login()
wandb.init(project ='vanillaRNN')

2024-05-14 06:46:35.671274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 06:46:35.671331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 06:46:35.672943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: cs23m066 (dlassignment). Use `wandb login --relogin` to force relogin


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
train_csv = "/kaggle/input/telugu/tel/tel_train.csv"
test_csv = "/kaggle/input/telugu/tel/tel_test.csv"
val_csv = "/kaggle/input/telugu/tel/tel_valid.csv"

In [4]:
train_data = pd.read_csv(train_csv, header=None)
train_input = train_data[0].to_numpy()
train_output = train_data[1].to_numpy()
val_data = pd.read_csv(val_csv,header = None)
val_input = val_data[0].to_numpy()
val_output = val_data[1].to_numpy()
test_data = pd.read_csv(test_csv,header= None)
print(len(train_input))
print(len(train_output))
print(len(val_input))
print(len(test_data))
print(val_input)
print(val_output)

51200
51200
4096
4096
['bheeshmudini' 'vinyasaanni' 'kaavachhunu' ... 'asramam' 'divine' 'dis']
['భీష్ముడిని' 'విన్యాసాన్ని' 'కావచ్చును' ... 'ఆశ్రమం' 'డివైన్' 'డిస్']


In [5]:
print(train_output[0][5]) #the size of input and output is 4096
maxi = 0
t =''
for x in val_input:
    maxi = max(maxi,len(x))
    if(maxi == len(x)):
        t=x
        
print(maxi,t)
t =''
maxi =0 
for x in val_output:
    maxi = max(maxi,len(x))
    if(maxi == len(x)):
        t=x
        
print(maxi,t)
    

ల
28 paramaanandabharithudayyaadu
19 పరమానందభరితుడయ్యాడు


In [6]:

def pre_processing(train_input,train_output):
    data = {
    "all_characters" : [],
    "char_num_map" : {},
    "num_char_map" : {},
    "source_charToNum": torch.zeros(len(train_input),30, dtype=torch.int, device=device),
    "source_data" : train_input,
        
    "all_characters_2" : [],
    "char_num_map_2" : {},
    "num_char_map_2" : {},
    "val_charToNum": torch.zeros(len(train_output),23, dtype=torch.int, device=device),
    "target_data" : train_output,
    "source_len" : 0,
    "target_len" : 0
 }
    k = 0 
    l = 0
    for i in range(0,len(train_input)):
        train_input[i] = "{" + train_input[i] + "}"*(29-len(train_input[i]))
        charToNum = []
        for char in (train_input[i]):
            index = 0
            if(char not in data["all_characters"]):
                data["all_characters"].append(char)
                index = data["all_characters"].index(char)
                data["char_num_map"][char] = index
                data["num_char_map"][index] = char
            else:
                index = data["all_characters"].index(char)
            
            charToNum.append(index)
            
        my_tensor = torch.tensor(charToNum,device = device)
        data["source_charToNum"][k] = my_tensor
        
        charToNum1 = []
        
        train_output[i] = "{" + train_output[i] + "}"*(22-len(train_output[i]))
        for char in (train_output[i]):
            index = 0
            if(char not in data["all_characters_2"]):
                data["all_characters_2"].append(char)
                index = data["all_characters_2"].index(char)
                data["char_num_map_2"][char] = index
                data["num_char_map_2"][index] = char
            else:
                index = data["all_characters_2"].index(char)
                
            charToNum1.append(index)
            
        my_tensor1 = torch.tensor(charToNum1,device = device)
        data["val_charToNum"][k] = my_tensor1
        
        k+=1
    
    data["source_len"] = len(data["all_characters"])
    data["target_len"] = len(data["all_characters_2"])
        
    return data
    
    
data = pre_processing(copy.copy(train_input),copy.copy(train_output))
# print(data["all_characters"])
# print(data["char_num_map"])
# print(data["num_char_map"])
# print(data["all_characters_2"])
# print(data["char_num_map_2"])
# print(data["num_char_map_2"])
print(data["source_charToNum"])
print(data['val_charToNum'])
print(data["num_char_map_2"])
print(data["num_char_map"])
print(train_input[0])
print(data['source_len'])
print(data['target_len'])


tensor([[ 0,  1,  2,  ...,  9,  9,  9],
        [ 0,  1,  2,  ...,  9,  9,  9],
        [ 0, 13,  2,  ...,  9,  9,  9],
        ...,
        [ 0,  1,  8,  ...,  9,  9,  9],
        [ 0,  3, 16,  ...,  9,  9,  9],
        [ 0, 14, 20,  ...,  9,  9,  9]], device='cuda:0', dtype=torch.int32)
tensor([[ 0,  1,  2,  ..., 10, 10, 10],
        [ 0,  1, 11,  ..., 10, 10, 10],
        [ 0, 14,  3,  ..., 10, 10, 10],
        ...,
        [ 0,  1, 25,  ..., 10, 10, 10],
        [ 0,  2, 20,  ..., 10, 10, 10],
        [ 0, 27, 25,  ..., 10, 10, 10]], device='cuda:0', dtype=torch.int32)
{0: '{', 1: 'వ', 2: 'ర', 3: '్', 4: 'గ', 5: 'ా', 6: 'ల', 7: 'ి', 8: 'న', 9: 'ే', 10: '}', 11: 'స', 12: 'త', 13: 'ద', 14: 'ఫ', 15: 'య', 16: 'క', 17: 'ట', 18: 'మ', 19: 'ో', 20: 'ూ', 21: 'ళ', 22: 'ప', 23: 'ధ', 24: 'ు', 25: 'ె', 26: 'ం', 27: 'చ', 28: 'ై', 29: 'డ', 30: 'ఖ', 31: 'ఉ', 32: 'ష', 33: 'ఆ', 34: 'ొ', 35: 'శ', 36: 'అ', 37: 'భ', 38: 'ృ', 39: 'ణ', 40: 'హ', 41: 'జ', 42: 'ీ', 43: 'ఇ', 44: 'బ', 45: 'ఐ', 46: 'ఒ', 47: 'ఎ

In [8]:
print(train_input[1])
print(train_output[1])

vastadira
వస్తాదిరా


In [9]:

def pre_processing_validation(val_input,val_output):
    data2 = {
    "all_characters" : [],
    "char_num_map" : {},
    "num_char_map" : {},
    "source_charToNum": torch.zeros(len(val_input),30, dtype=torch.int, device=device),
    "source_data" : val_input,
    "all_characters_2" : [],
    "char_num_map_2" : {},
    "num_char_map_2" : {},
    "val_charToNum": torch.zeros(len(val_output),23, dtype=torch.int, device=device),
    "target_data" : val_output,
    "source_len" : 0,
    "target_len" : 0
 }
    k = 0 
    l = 0
    
    m1 = data["char_num_map"]
    m2 = data["char_num_map_2"]
    
    for i in range(0,len(val_input)):
        val_input[i] = "{" + val_input[i] + "}"*(29-len(val_input[i]))
        charToNum = []
        for char in (val_input[i]):
            index = 0
            if(char not in data2["all_characters"]):
                data2["all_characters"].append(char)
                index = m1[char]
                data2["char_num_map"][char] = index
                data2["num_char_map"][index] = char
            else:
                index = m1[char]
            
            charToNum.append(index)
            
        my_tensor = torch.tensor(charToNum,device = device)
        data2["source_charToNum"][k] = my_tensor
        
        charToNum1 = []
        val_output[i] = "{" + val_output[i] + "}"*(22-len(val_output[i]))
        for char in (val_output[i]):
            index = 0
            if(char not in data2["all_characters_2"]):
                data2["all_characters_2"].append(char)
                index = m2[char]
                data2["char_num_map_2"][char] = index
                data2["num_char_map_2"][index] = char
            else:
                index = m2[char]
                
            charToNum1.append(index)
            
        my_tensor1 = torch.tensor(charToNum1,device = device)
        data2["val_charToNum"][k] = my_tensor1
        
        k+=1
    
    data2["source_len"] = len(data2["all_characters"])
    data2["target_len"] = len(data2["all_characters_2"])
        
    return data2
    
    
data2 = pre_processing_validation(copy.copy(val_input),copy.copy(val_output))
# print(data["all_characters"])
# print(data["char_num_map"])
# print(data["num_char_map"])
# print(data["all_characters_2"])
# print(data["char_num_map_2"])
# print(data["num_char_map_2"])
print(data2["num_char_map"])
print(data2["source_charToNum"].shape)

print(data2["num_char_map_2"])
print(data2['val_charToNum'][0])


print(val_input[0])
print(data2['source_len'])
print(data2['target_len'])


{0: '{', 23: 'b', 20: 'h', 8: 'e', 10: 's', 15: 'm', 17: 'u', 12: 'd', 6: 'i', 7: 'n', 9: '}', 1: 'v', 22: 'y', 2: 'a', 21: 'k', 14: 'c', 11: 't', 3: 'r', 19: 'p', 5: 'l', 16: 'o', 4: 'g', 24: 'j', 18: 'w', 26: 'x', 13: 'f', 25: 'z', 27: 'q'}
torch.Size([4096, 30])
{0: '{', 37: 'భ', 42: 'ీ', 32: 'ష', 3: '్', 18: 'మ', 24: 'ు', 29: 'డ', 7: 'ి', 8: 'న', 10: '}', 1: 'వ', 15: 'య', 5: 'ా', 11: 'స', 16: 'క', 27: 'చ', 12: 'త', 2: 'ర', 26: 'ం', 22: 'ప', 6: 'ల', 20: 'ూ', 49: 'థ', 33: 'ఆ', 35: 'శ', 40: 'హ', 19: 'ో', 4: 'గ', 41: 'జ', 13: 'ద', 34: 'ొ', 28: 'ై', 9: 'ే', 46: 'ఒ', 25: 'ె', 17: 'ట', 39: 'ణ', 43: 'ఇ', 38: 'ృ', 54: 'ఓ', 23: 'ధ', 45: 'ఐ', 47: 'ఎ', 36: 'అ', 44: 'బ', 52: 'ఏ', 14: 'ఫ', 31: 'ఉ', 30: 'ఖ', 21: 'ళ', 51: 'ఊ', 48: 'ౌ', 55: 'ఔ', 57: 'ఠ', 58: 'ఘ', 56: 'ఞ', 50: 'ఈ', 59: 'ఛ', 62: 'ఋ', 60: 'ః', 53: 'ఢ'}
tensor([ 0, 37, 42, 32,  3, 18, 24, 29,  7,  8,  7, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10], device='cuda:0', dtype=torch.int32)
bheeshmudini
28
62


In [10]:
class MyDataset(Dataset):
    def __init__(self, x,y):
        self.source = x
        self.target = y
    
    def __len__(self):
        return len(self.source)
    
    def __getitem__(self, idx):
        source_data = self.source[idx]
        target_data = self.target[idx]
        return source_data, target_data

In [11]:
class MyDataset2(Dataset):
    def __init__(self, x,y):
        self.source = x
        self.target = y
    
    def __len__(self):
        return len(self.source)
    
    def __getitem__(self, idx):
        source_data = self.source[idx]
        target_data = self.target[idx]
        return source_data, target_data

In [30]:
def validationAccuracy(encoder,decoder,batchsize,tf_ratio):
    
    dataLoader = dataLoaderFun("validation",batchsize) # dataLoader depending on train or validation
    
    encoder.eval()
    decoder.eval()
    
    validation_accuracy = 0
    validation_loss = 0
    
    lossFunction = nn.NLLLoss()
    
    for batch_num, (source_batch, target_batch) in enumerate(dataLoader):
        
        encoder_initial_state = encoder.getInitialState() #hiddenlayers * BatchSize * Neurons
        encoder_output, encoder_current_state = encoder(source_batch,encoder_initial_state)
        #print(encoder_output)
        #success till here

        loss = 0 # decoder starts form here
        correct = 0

        output_seq_len = target_batch.shape[1] # here you will get as name justified. 40

        decoder_actual_output = []
        #print(target_batch)

        randNumber = random.random()

        decoder_curr_state = encoder_current_state

        for i in range(0,output_seq_len):

            if(i == 0):
                decoder_input_tensor = target_batch[:, i].reshape(batchsize,1) #32*1
                #print(dec_input_tensor.shape)
            else:
                if randNumber < tf_ratio:
                    decoder_input_tensor = target_batch[:, i].reshape(batchsize, 1) # current batch is passed
                else:
                    decoder_input_tensor = decoder_input_tensor.reshape(batchsize, 1) # prev result is passed

            #print(curr_target_chars.shape) #32
            decoder_output, decoder_curr_state = decoder(decoder_input_tensor,decoder_curr_state)
            #print(decoder_output.shape) #(32*1*67) but your output is (32*1*65) becz ur output size is 65
            topv, topi = decoder_output.topk(1)  # you will get top vales and their indices.
            #print("topv", topv)
            decoder_input_tensor = topi.squeeze().detach()  # here whatever top softmax indeces are present but converted to 1 dimension
            #print(decoder_input_tensor.shape)
            decoder_actual_output.append(decoder_input_tensor) # softmax values are attached                    

            decoder_output = decoder_output[:, -1, :] #it is just reduce the size from (32*1*67) to (32*67)
            #print(decoder_output.shape,curr_target_chars.shape)
            #print(decoder_output.shape,curr_target_chars.shape)

            curr_target_chars = target_batch[:, i] #(32)
            curr_target_chars = curr_target_chars.type(dtype=torch.long)
            #print(curr_target_chars)

            loss+=(lossFunction(decoder_output, curr_target_chars)) # you are passing 32*67 softmax values to curr_target_chars which has the 32*1

        tensor_2d = torch.stack(decoder_actual_output)
        decoder_actual_output = tensor_2d.t() #it is outside the for loop

        validation_accuracy += (decoder_actual_output == target_batch).all(dim=1).sum().item() # it is simple just summing up the equal values
        validation_loss += (loss.item()/output_seq_len)

        if(batch_num%20 == 0):
            print("bt:", batch_num, " loss:", loss.item()/output_seq_len)
        #'k'/24
        # here you get the actual word letters seqeunces softamx indeces
        #[[0,1,2],[0,1,2]] = [shr,ram] 32*40
        #correct = (decoder_actual_output == target_batch).all(dim=1).sum().item()
        #accuracy = accuracy + correct
    
    encoder.train()
    decoder.train()
    print("validation_accuracy",validation_accuracy/40.96)
    print("validation_loss",validation_loss)
    wandb.log({'validation_accuracy':validation_accuracy/40.96})
    wandb.log({'validation_loss':validation_loss})

In [14]:
class Encoder(nn.Module):
    
    def __init__(self,inputDim,embSize,encoderLayers,hiddenLayerNuerons,cellType,batch_size):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(inputDim, embSize)
        self.encoderLayers = encoderLayers
        self.hiddenLayerNuerons = hiddenLayerNuerons
        self.batch_size = batch_size
        
        if(cellType=='GRU'):
            self.rnn = nn.GRU(embSize,hiddenLayerNuerons,num_layers=encoderLayers, batch_first=True)
            
    def forward(self, currentInput, prevState):
        embdInput = self.embedding(currentInput)
        output, prev_state = self.rnn(embdInput, prevState)
        return output, prev_state
    
    def getInitialState(self):
        return torch.zeros(self.encoderLayers,self.batch_size,self.hiddenLayerNuerons, device=device)
    
class Decoder(nn.Module):
    def __init__(self,outputDim,embSize,hiddenLayerNuerons,decoderLayers,cellType):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(outputDim, embSize)
        if(cellType=="GRU"):
            self.rnn = nn.GRU(embSize,hiddenLayerNuerons,num_layers=decoderLayers, batch_first=True)
        self.fc = nn.Linear(hiddenLayerNuerons, outputDim) # it is useful for mapping the calculation to vocabularu
        self.softmax = nn.LogSoftmax(dim=2) #output is in 3rd column 

    def forward(self, current_input, prev_state):
        embd_input = self.embedding(current_input)
        curr_embd = F.relu(embd_input)
        output, prev_state = self.rnn(curr_embd, prev_state)
        output = self.softmax(self.fc(output)) 
        return output, prev_state 

In [27]:
input_dim = data["source_len"]
output_dim = data["target_len"]
char_embd_dim=64
hidden_layer_neurons = 512
learning_rate  =0.0001
batch_size = 64
number_of_layers = 10
tf_ratio = 0.2
epochs = 50
#train(64,1,1,512,'GRU','NO',0.4,20,64,1e-4,"Adam",0.2)

bt: 0  loss: 4.114956067956013


KeyboardInterrupt: 

In [15]:
data = pre_processing(copy.copy(train_input),copy.copy(train_output))

In [17]:
def dataLoaderFun(dataName,batch_size):
    if(dataName == 'train'):
        dataset = MyDataset(data["source_charToNum"],data['val_charToNum'])
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
    else:
        dataset = MyDataset(data2["source_charToNum"],data2['val_charToNum'])
        return  DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [31]:
def train(embSize,encoderLayers,decoderLayers,hiddenLayerNuerons,cellType,bidirection,dropout,epochs,batchsize,learningRate,optimizer,tf_ratio):
    #add optimizer,tf_ratio to wandb parameters
    
    dataLoader = dataLoaderFun("train",batchsize) # dataLoader depending on train or validation
    
    
    encoder = Encoder(data["source_len"],embSize,encoderLayers,hiddenLayerNuerons,cellType,batchsize).to(device)
    decoder = Decoder(data["target_len"],embSize,hiddenLayerNuerons,encoderLayers,cellType).to(device)
    
    # done till here
    if(optimizer == 'Adam'):
        encoderOptimizer = optim.Adam(encoder.parameters(), lr=learningRate)
        decoderOptimizer = optim.Adam(decoder.parameters(), lr=learningRate)
    else:
        encoderOptimizer = optim.NAdam(encoder.parameters(), lr=learningRate)
        decoderOptimizer = optim.NAdam(decoder.parameters(), lr=learningRate)
    
    lossFunction = nn.NLLLoss()

    for epoch in range (0,epochs):
    
        train_accuracy = 0 
        train_loss = 0 

        for batch_num, (source_batch, target_batch) in enumerate(dataLoader):
                        
            encoder_initial_state = encoder.getInitialState() #hiddenlayers * BatchSize * Neurons
            
            encoder_output, encoder_current_state = encoder(source_batch,encoder_initial_state)
            #print(encoder_output)
            #success till here
            
            loss = 0 # decoder starts form here
            correct = 0
            
            output_seq_len = target_batch.shape[1] # here you will get as name justified. 40

            decoder_actual_output = []
            #print(target_batch)
            
            randNumber = random.random()

            decoder_curr_state = encoder_current_state

            for i in range(0,output_seq_len):
                
                if(i == 0):
                    decoder_input_tensor = target_batch[:, i].reshape(batchsize,1) #32*1
                    #print(dec_input_tensor.shape)
                else:
                    if randNumber < tf_ratio:
                        decoder_input_tensor = target_batch[:, i].reshape(batchsize, 1) # current batch is passed
                    else:
                        decoder_input_tensor = decoder_input_tensor.reshape(batchsize, 1) # prev result is passed

                #print(curr_target_chars.shape) #32
                decoder_output, decoder_curr_state = decoder(decoder_input_tensor,decoder_curr_state)
                #print(decoder_output.shape) #(32*1*67) but your output is (32*1*65) becz ur output size is 65
                topv, topi = decoder_output.topk(1)  # you will get top vales and their indices.
                #print("topv", topv)
                decoder_input_tensor = topi.squeeze().detach()  # here whatever top softmax indeces are present but converted to 1 dimension
                #print(decoder_input_tensor.shape)
                decoder_actual_output.append(decoder_input_tensor) # softmax values are attached                    
                        
                decoder_output = decoder_output[:, -1, :] #it is just reduce the size from (32*1*67) to (32*67)
                #print(decoder_output.shape,curr_target_chars.shape)
                #print(decoder_output.shape,curr_target_chars.shape)

                curr_target_chars = target_batch[:, i] #(32)
                curr_target_chars = curr_target_chars.type(dtype=torch.long)
                #print(curr_target_chars)
                
                loss+=(lossFunction(decoder_output, curr_target_chars)) # you are passing 32*67 softmax values to curr_target_chars which has the 32*1
                
            tensor_2d = torch.stack(decoder_actual_output)
            decoder_actual_output = tensor_2d.t() #it is outside the for loop
            #print(decoder_actual_output) #32*40
            if(batch_num == 0 and epoch == epochs-1):
                numToCharConverter(target_batch,decoder_actual_output,data) 
                
            train_accuracy += (decoder_actual_output == target_batch).all(dim=1).sum().item() # it is simple just summing up the equal values
            train_loss += (loss.item()/output_seq_len)
            
            if(batch_num%200 == 0):
                print("bt:", batch_num, " loss:", loss.item()/output_seq_len)
            #'k'/24
            # here you get the actual word letters seqeunces softamx indeces
            #[[0,1,2],[0,1,2]] = [shr,ram] 32*40
            #correct = (decoder_actual_output == target_batch).all(dim=1).sum().item()
            #accuracy = accuracy + correct
            encoderOptimizer.zero_grad()
            decoderOptimizer.zero_grad()
            loss.backward()
            encoderOptimizer.step()
            decoderOptimizer.step()
            
        print("train_accuracy",train_accuracy/512)
        print("train_loss",train_loss)
        wandb.log({'train_accuracy':train_accuracy/512})
        wandb.log({'train_loss':train_loss})
        validationAccuracy(encoder,decoder,batchsize,tf_ratio)

In [19]:
def numToCharConverter(inputArray,outputArray,data):
    mp = data['num_char_map_2']
    t1 = ''
    t2 = ''
    for row1, row2 in zip(inputArray,outputArray):
        t1=''
        t2=''
        for e1, e2 in zip(row1,row2):
            t1+=mp[e1.item()]
            t2+=mp[e2.item()]
        print(t1," ",t2)
            
        
    

In [ ]:
def main_fun():
    wandb.init(project ='vanillaRNN')
    params = wandb.config
    with wandb.init(project = 'vanillaRNN', name='embedding'+str(params.embSize)+'cellType'+params.cellType+'batchSize'+str(params.batchsize)) as run:
        train(params.embSize,params.encoderLayers,params.decoderLayers,params.hiddenLayerNuerons,params.cellType,params.bidirection,params.dropout,params.epochs,params.batchsize,params.learningRate,params.optimizer,params.tf_ratio)
    
sweep_params = {
    'method' : 'bayes',
    'name'   : 'DeepLearningAssignment3',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'validation_accuracy',
    },
    'parameters' : {
        'embSize':{'values':[16,32,64]},
        'encoderLayers':{'values':[1,5,10]},
        'decoderLayers' : {'values' : [1,5,10]},
        'hiddenLayerNuerons'   : {'values' : [64,256,512]},
        'cellType' : {'values' : ['GRU'] } ,
        'bidirection' : {'values' : ['no']},
        'dropout' : {'values' : [0.2,0.3]},
        'epochs'  : {'values': [10,30,50]},
        'batchsize' : {'values' : [32,64]},
        'learningRate' : {'values' : [1e-2,1e-3,1e-4]},
        'optimizer':{'values' : ['Adam','Nadam']},
        'tf_ratio' :{'values' : [0.2,0.4,0.5]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'vanillaRNN')
wandb.agent(sweepId,function =main_fun,count = 2)
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: p7xk0nii
Sweep URL: https://wandb.ai/dlassignment/vanillaRNN/sweeps/p7xk0nii
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: 5hweqpw6 with config:
wandb: 	batchsize: 64
wandb: 	bidirection: no
wandb: 	cellType: GRU
wandb: 	decoderLayers: 10
wandb: 	dropout: 0.2
wandb: 	embSize: 64
wandb: 	encoderLayers: 1
wandb: 	epochs: 30
wandb: 	hiddenLayerNuerons: 64
wandb: 	learningRate: 0.001
wandb: 	optimizer: Adam
wandb: 	tf_ratio: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status
    self._loop_check_status(
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 224, in _loop_check_status
    local_handle = request()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sd

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


bt: 0  loss: 4.226894544518513
bt: 200  loss: 1.8207030918287195
bt: 400  loss: 1.8083355115807576
bt: 600  loss: 1.6731506015943445
train_accuracy 1.880859375
train_loss 1350.1659686047105
bt: 0  loss: 1.4505419523819634
bt: 20  loss: 1.5248487721318784
bt: 40  loss: 1.4063297769297725
bt: 60  loss: 0.3964451084966245
validation_accuracy 6.54296875
validation_loss 84.27477210500967
bt: 0  loss: 1.7409896850585938
bt: 200  loss: 1.4113021518873132
bt: 400  loss: 1.557510873545771
bt: 600  loss: 1.5796760890794836
train_accuracy 14.2265625
train_loss 1034.0812792985319
bt: 0  loss: 1.1592445373535156
bt: 20  loss: 0.12331232817276665
bt: 40  loss: 1.1536732549252717
bt: 60  loss: 1.1071439825970193
validation_accuracy 12.4755859375
validation_loss 64.11064842472905
bt: 0  loss: 1.400758992070737
bt: 200  loss: 1.397218123726223
bt: 400  loss: 1.4387178835661516
bt: 600  loss: 1.4976675614066746
train_accuracy 19.173828125
train_loss 927.5677470668496
bt: 0  loss: 1.0188982590385105
bt: 

train_accuracy,▁▅▇▇▆▇▇▇▆▇█▇▆▇▇▇▇█▇▇▇▇▇▇█▇█▇▇█
train_loss,█▅▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▃▄▇▆▃▇▇▄▄▅▅▄▇▇▅▄▇▇▅█▅▆▅▄▆▆▄▄▆
validation_loss,█▅▄▃▃▄▂▂▃▃▂▂▃▂▂▂▂▁▁▂▁▂▂▂▂▁▁▂▂▁
train_accuracy,21.38086
train_loss,686.75206
validation_accuracy,21.94824
validation_loss,41.48012


wandb: Agent Starting Run: 7zysbfrf with config:
wandb: 	batchsize: 64
wandb: 	bidirection: no
wandb: 	cellType: GRU
wandb: 	decoderLayers: 10
wandb: 	dropout: 0.2
wandb: 	embSize: 64
wandb: 	encoderLayers: 10
wandb: 	epochs: 50
wandb: 	hiddenLayerNuerons: 512
wandb: 	learningRate: 0.001
wandb: 	optimizer: Adam
wandb: 	tf_ratio: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


bt: 0  loss: 4.138681494671365
bt: 200  loss: 1.5060567441193953
bt: 400  loss: 1.589910424273947
bt: 600  loss: 1.4415138908054517
train_accuracy 0.0
train_loss 1237.5827971748677
bt: 0  loss: 1.1126754594885784
bt: 20  loss: 1.2093979379405146
bt: 40  loss: 1.156596556953762
bt: 60  loss: 1.2054286625074304
validation_accuracy 0.0
validation_loss 72.85178574271825
bt: 0  loss: 1.339441962864088
bt: 200  loss: 1.3588578597359036
bt: 400  loss: 1.3025303716244905
bt: 600  loss: 1.232225749803626
train_accuracy 0.0
train_loss 1069.9040426171346
bt: 0  loss: 1.0503410671068274
bt: 20  loss: 0.8863300655199133
bt: 40  loss: 0.990118524302607
bt: 60  loss: 0.915020403654679
validation_accuracy 0.0
validation_loss 60.11573866139287
bt: 0  loss: 1.257032062696374
bt: 200  loss: 1.2118649690047554
bt: 400  loss: 1.0629578466000764
bt: 600  loss: 0.97290097112241
train_accuracy 0.119140625
train_loss 860.2513406173042
bt: 0  loss: 0.6160142732703168
bt: 20  loss: 0.6024523610654084
bt: 40  los

In [ ]:

train(data)